In [1]:
from stochastic_volatility_models.src.models.heston import HestonModel

In [2]:
model = HestonModel(
	parameters={
		"initial_variance": 0.05,
		"long_term_volatility": 0.05,
		"volatility_of_volatility": 0.3,
		"mean_reversion_rate": 2.0,
		"wiener_correlation": -0.5,
	}
)

In [4]:
model.parameters

{'initial_variance': 0.05,
 'long_term_volatility': 0.05,
 'volatility_of_volatility': 0.3,
 'mean_reversion_rate': 2.0,
 'wiener_correlation': -0.5}